In [1]:
!pip install networkx

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import ast

PATH = '..\\Final_Persons_And_Recommenders.csv'
df = pd.read_csv(PATH)
df['Recommenders ID'] = df['Recommenders ID'].apply(ast.literal_eval)
df.shape

(964, 2)

In [10]:
# Create a new binary column 'recur_endorse' initialized to 0
df['recur_endorse'] = 0

# Create a set to store recursive endorsements
recursion_found = set()

# Create a dictionary to map IDs to their recommenders
recommender_dict = df.set_index('ID')['Recommenders ID'].to_dict()

# Check for recursive recommendations
for id, recommenders in recommender_dict.items():
    for recommender in recommenders:
        if recommender in recommender_dict and id in recommender_dict[recommender]:
            recursion_found.add(tuple(sorted((id, recommender))))

# Update the 'recur_endorse' column for the found recursive endorsements
for id1, id2 in recursion_found:
    df.loc[df['ID'] == id1, 'recur_endorse'] = 1
    df.loc[df['ID'] == id2, 'recur_endorse'] = 1

# Print the number of recursive recommendations found
print(len(recursion_found))
# print("Recursive Recommendations: ", )
# for id1, id2 in recursion_found:
#     print(f"ID {id1} recommends ID {id2} and vice versa.")

45


In [11]:
df.columns.to_list()

['ID', 'Recommenders ID', 'recur_endorse']

In [12]:
df['recur_endorse'].value_counts()

recur_endorse
0    876
1     88
Name: count, dtype: int64

In [13]:
# Save the updated DataFrame to a CSV file
df.to_csv('../Final_Resume_Texts/dataset_recur_endorse.csv', index=False)